In [1]:
import random
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from deap import base,creator, tools

In [2]:
dataset = pd.read_csv('dataset.csv', delimiter=';', low_memory='False')

In [3]:
#Drop columns that are not needed
dataset = dataset.drop(['user','gender','age','how_tall_in_meters','weight','body_mass_index'], axis=1)

In [4]:

#Centering
scaler = StandardScaler(with_mean=True, with_std=False)
dataset[['x1','y1','z1','x2','y2','z2','x3','y3','z3','x4','y4','z4']] = scaler.fit_transform(dataset[['x1','y1','z1','x2','y2','z2','x3','y3','z3','x4','y4','z4']])

#MinMax Scaling
scaler = MinMaxScaler();
dataset[['x1','y1','z1','x2','y2','z2','x3','y3','z3','x4','y4','z4']] = scaler.fit_transform(dataset[['x1','y1','z1','x2','y2','z2','x3','y3','z3','x4','y4','z4']])



In [5]:
#Getting the means
means = dataset.groupby('Class')[['x1','y1','z1','x2','y2','z2','x3','y3','z3','x4','y4','z4']].mean()

In [6]:
#Separating means for the sitting state and the other states
sitting_mean = means.T.pop('sitting').T
states_means = means.T.drop('sitting', axis=1).T

sitting_mean

x1    0.366705
y1    0.419138
z1    0.545830
x2    0.449931
y2    0.568754
z2    0.716909
x3    0.519283
y3    0.581117
z3    0.505974
x4    0.827674
y4    0.680005
z4    0.759148
Name: sitting, dtype: float64

In [7]:
states_means

,x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,y4,z4
Class,,,,,,,,,,,,
sittingdown,0.373142,0.457112,0.480843,0.414011,0.555831,0.608343,0.504431,0.598439,0.511138,0.753158,0.702511,0.801967
standing,0.367495,0.458642,0.489369,0.491653,0.702995,0.638789,0.518889,0.600316,0.513219,0.760258,0.720067,0.768590
standingup,0.368815,0.450662,0.476264,0.404896,0.544285,0.593578,0.488990,0.550286,0.496797,0.735012,0.708399,0.775540
walking,0.365761,0.462549,0.480816,0.312931,0.447328,0.413280,0.509756,0.624089,0.504906,0.749150,0.729366,0.750483


In [8]:
states_means = np.array(states_means)
sitting_mean = np.array(sitting_mean)


In [9]:
#Making the genetic algorithm

w = []
for _ in range(0,12):
    w.append(1.0)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)


toolbox = base.Toolbox()
toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, n=12)
                 
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


def evaluate(v):
    c=0.2
    
    v= np.array(v).reshape(1,-1)
    other_states_sum = 0
    
    for state in states_means:
        other_states_sum += cosine_similarity(v,state.reshape(1,-1))
    
    f = (cosine_similarity(v,sitting_mean.reshape(1,-1)) + c*(1 - 0.25 * other_states_sum)) / (1 + c)
    
    return f[0]


toolbox.register("mate", tools.cxSimulatedBinary, eta=0.1)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)


    

In [10]:
def ga():
    pop = toolbox.population(n=20)
    CXPB, MUTPB, NGEN = 0.6, 0, 100

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    
    
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        
        print("\n----GEN "+ str(g+1) +"----\n")
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring =list(map(toolbox.clone, offspring))
        
        
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            if random.random() < CXPB:
                toolbox.mate(child1,child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)

        
    

In [11]:
ga()



----GEN 1----

  Min 0.04253813504229762
  Max 0.7816862954960997
  Avg 0.6848613036476535
  Std 0.1660319786674022

----GEN 2----

  Min 0.5975522941921552
  Max 0.7816862954960997
  Avg 0.7246838469590738
  Std 0.06428902848363632

----GEN 3----

  Min 0.22800036740496535
  Max 0.7695470988390002
  Avg 0.639298921198295
  Std 0.18203394744441337

----GEN 4----

  Min 0.08052689848915238
  Max 0.7695470988390003
  Avg 0.6454963164161309
  Std 0.2091217382436841

----GEN 5----

  Min 0.061044592071262095
  Max 0.7695470988390003
  Avg 0.669892433050826
  Std 0.16695824546746393

----GEN 6----

  Min 0.13068538563410784
  Max 0.7707840740727869
  Avg 0.6910416462879827
  Std 0.14827926855612453

----GEN 7----

  Min 0.757058140182063
  Max 0.7707840740727869
  Avg 0.7655296311995203
  Std 0.004153728501425078

----GEN 8----

  Min 0.6632548964284904
  Max 0.7795754892264349
  Avg 0.7579507630597453
  Std 0.029106216207248257

----GEN 9----

  Min 0.2593041490636655
  Max 0.779575489226

  Min 0.8168197433787047
  Max 0.8256516163391073
  Avg 0.8242533545533638
  Std 0.002380513414199114

----GEN 71----

  Min 0.8254833381633889
  Max 0.8256516163391074
  Avg 0.8255842137504453
  Std 4.9597298997210664e-05

----GEN 72----

  Min 0.825448045433492
  Max 0.8257594984791665
  Avg 0.8256163385078535
  Std 5.90282487842509e-05

----GEN 73----

  Min 0.8255862843215418
  Max 0.8257594984791665
  Avg 0.8256462160288762
  Std 4.272156807238482e-05

----GEN 74----

  Min 0.8254968038826179
  Max 0.8257594984791665
  Avg 0.8256757107473861
  Std 6.741707740995423e-05

----GEN 75----

  Min 0.8253829418094862
  Max 0.8258826606063819
  Avg 0.8257037591296065
  Std 0.0001016624561240386

----GEN 76----

  Min 0.8256678871473196
  Max 0.8259261185671171
  Avg 0.8257555686441151
  Std 6.745023658865292e-05

----GEN 77----

  Min 0.8252745252252462
  Max 0.8259776757685965
  Avg 0.8257629461600832
  Std 0.00017765137565193718

----GEN 78----

  Min 0.8245998389836213
  Max 0.82605557